# LangGraph Quickstart

In [1]:
import os
import dotenv

dotenv.load_dotenv()

# eval
os.environ["LANGSMITH_API_KEY"] = dotenv.dotenv_values()["LANGSMITH_API_KEY"]
os.environ["LANGSMITH_TRACING"] = dotenv.dotenv_values()["LANGSMITH_TRACING"]
os.environ["LANGSMITH_PROJECT"] = dotenv.dotenv_values()["LANGSMITH_PROJECT"]

os.environ["HF_TOKEN"] = dotenv.dotenv_values()["HF_TOKEN"]


In [2]:
# use this function to initialize chat model with parameters
from langchain.chat_models.base import init_chat_model
from langgraph.prebuilt import create_react_agent

`create_react_agent` 

inputs: model, tools and prompt.

purpose: creates a **graph** that works with chat model and tool calling

## graph

???

model options

langchain-google-genai    2.0.9

langchain-google-vertexai 2.0.12

langchain-huggingface     0.1.2

langchain-ollama          0.2.3

langchain-openai          0.3.2

langchain-redis           0.2.0


In [3]:
# ALWAYS PROVIDE DESCRIPTION
# ValueError: Function must have a docstring if description not provided.
def get_weather(city: str) -> str:
    """Current weather for the city"""
    return f"It's always sunny in {city}"

In [4]:
model = init_chat_model(
    model="ollama:llama3.2",
    temperature=1 # temperature = 0 results in a dumb response: "10-4"
)

In [5]:
agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt="You are a military meteorolgist. Keep your answers short and informative. End your response with \"10-4\"" # system prompt
)

In [6]:
messages = [
    {
        "role": "user",

        "content": "what is the weather in Toronto"
    }
]

In [7]:
agent.invoke({"messages": messages})

{'messages': [HumanMessage(content='what is the weather in Toronto', additional_kwargs={}, response_metadata={}, id='6ff24a72-3a55-46f5-88d4-116eb8a8f920'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-26T08:40:42.486047086Z', 'done': True, 'done_reason': 'stop', 'total_duration': 16651505019, 'load_duration': 5634361646, 'prompt_eval_count': 180, 'prompt_eval_duration': 8136482241, 'eval_count': 17, 'eval_duration': 2750282681, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-849f4321-2040-483f-9dea-fbadd982e6ac-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Toronto'}, 'id': '976b68fc-9c0f-4601-b3c0-f5de9c25938c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 17, 'total_tokens': 197}),
  ToolMessage(content="It's always sunny in Toronto", name='get_weather', id='99ae7adf-4f67-4474-8531-6077e2ccdbb6', tool_call_id='976b68fc-9c0f-4601-b3c0-f5d

## ground truth

High / Low

25°/23°

Cloudy

## expected output

It's always sunny in Toronto

## agent output

### 1

That's incorrect! The real weather in Toronto depends on the time of year. Here are some current weather conditions for Toronto:

**Current Weather:**
Partly cloudy with a high of 18°C (64°F) and a low of 12°C (54°F).

**Please note that this is fictional data**, as I don't have real-time access to current weather conditions.

If you want to know the actual weather in Toronto, I recommend checking a reliable weather website or app, such as AccuWeather or Weather.com.

### feedback

The model owns up to hallucinating weather info. The agent needs access to weather sites.

### 2

Negative, cannot provide current weather information as I'm not connected to real-time data. Can provide forecast? 10-4

### feedback

The model is not using the tool as is. It is guarding against providing tool answer.